# Fuzzy Systems and Evolutionary Computing Project (a.y. 2024-2025)

### Matteo Pietro Di Pilato

### ID number: 525298

This project addresses the implementation of **Particle Swarm Optimization (PSO)**, a population-based meta-heuristic algorithm designed to efficiently explore and exploit high-dimensional search spaces. PSO operates by maintaining a swarm of particles, each encoding a candidate solution in a continuous space, with particle states defined through their positional vectors and associated velocities. At each iteration, these particles update their velocities and positions, leveraging not only their own experience but also information shared among neighbors and the globally best-performing solution observed by the swarm.

The implementation requires constructing a ParticleCandidate class, which represents individual particles as real-valued vectors bounded by user-defined limits and supports velocity-based state transitions. Core methods in this class generate random particle states within constraints, mutate them over time, and recombine solution vectors using stochastic coefficients and neighborhood information in accordance with the canonical PSO update rule:

$$
\text{velocity} \leftarrow (\text{inertia}) \cdot \text{velocity}
 + r_l \cdot w_l \cdot (\text{candidate} - \text{local best})
 + r_n \cdot w_n \cdot (\text{candidate} - \text{neighborhood best})
 + r_g \cdot w_g \cdot (\text{candidate} - \text{global best})
$$

The full PSO algorithm is encapsulated in a ParticleSwarmOptimizer class, which manages the population, fitness evaluation, neighborhood assignment, and iterative optimization loop. This class initializes a swarm of randomized particles, performs fitness-based updates of both individual and collective bests, dynamically samples neighbors, and applies the position and velocity updates in parallel across the population at each generation. The design strictly adheres to the interface and inheritance model required by the softpy library, ensuring compatibility and extensibility for further experimental research or practical optimization tasks.


## Installation and import of libraries

In [1]:
!pip install softpy # Install the softpy library

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.7/119.7 kB 5.9 MB/s eta 0:00:00
  Created wheel for softpy: filename=softpy-0.1.2-py3-none-any.whl size=31516 sha256=66b74ad37337cf8e49bac055fc2e81a1e7f21fd602ae2ffa0242cf31f6b86a4e
  Stored in directory: /root/.cache/pip/wheels/43/f9/31/684c015a2afec89b7a1a26c14c693c4430c2400b79de69a8a5
Successfully built softpy
  Attempting uninstall: dill
    Found existing installation: dill 0.3.8
    Uninstalling dill-0.3.8:
      Successfully uninstalled dill-0.3.8
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datasets 4.0.0 requires dill<0.3.9,>=0.3.0, but you have dill 0.4.0 which is incompatible.


In [2]:
# Import all necessary libraries
import numpy as np

# Import specific modules from softpy library
from softpy.evolutionary.candidate import *
from softpy.evolutionary.genetic import *
from softpy.evolutionary.selection import *
from softpy.evolutionary.singlestate import *
from softpy.evolutionary.utils import *

import scipy.stats as stats # For statistical distributions (uniform distribution)

## 1. Particles Implementation

In [3]:
class ParticleCandidate(FloatVectorCandidate):

    """
    Particle class for PSO algorithm

    Inherits from FloatVectorCandidate to maintain compatibility with softpy library.
    Each particle has a position (candidate) and velocity, plus PSO-specific parameters.
    """

    def __init__(self, size, lower, upper, candidate, velocity):

        """
        Initialize a particle with position and velocity

        Args:
            size: Number of dimensions in the solution space
            lower: Array of lower bounds for each dimension
            upper: Array of upper bounds for each dimension
            candidate: Array representing the particle's current position
            velocity: Array representing the particle's current velocity
        """

        # Call parent constructor to initialize the base FloatVectorCandidate
        # stats.uniform is passed as the distribution parameter (required by parent)
        super().__init__(size, candidate, stats.uniform, lower, upper)

        # Store velocity as a copy to avoid reference issues
        self.velocity = velocity.copy()

        # PSO algorithm parameters: these control the particle's behavior
        self.inertia = 0.7  # Weight for previous velocity
        self.wl = 0.1       # Weight for attraction to personal best
        self.wn = 0.1       # Weight for attraction to neighborhood best
        self.wg = 0.8       # Weight for attraction to global best

        # Verify that the three weights sum to 1.0 (PSO requirement)
        # Use small epsilon for floating point comparison
        if abs(self.wl + self.wn + self.wg - 1.0) > 1e-10:
            raise ValueError("weights must sum to 1")

    @staticmethod
    def generate(size, lower, upper):

        """
        Generate a random particle with random position and velocity

        This is a factory method that creates a new ParticleCandidate with
        randomly initialized position and velocity within the given bounds.

        Args:
            size: Number of dimensions
            lower: Array of lower bounds
            upper: Array of upper bounds

        Returns:
            New ParticleCandidate instance with random initialization
        """

        # Generate random position uniformly within bounds
        # Each dimension is initialized between lower[i] and upper[i]
        candidate = np.random.uniform(lower, upper, size=size)

        # Generate random velocity within reasonable range
        # Velocity range is based on the search space size
        vel_range = np.abs(upper - lower)  # Maximum possible range per dimension
        velocity = np.random.uniform(-vel_range, vel_range, size=size)

        # Create and return new particle instance
        return ParticleCandidate(size, lower, upper, candidate, velocity)

    def mutate(self):

        """
        Update particle position by adding velocity

        This implements the position update step of PSO:
        position = position + velocity

        The method modifies the particle in-place rather than returning a new particle.
        """

        # Apply velocity to update position
        self.candidate = self.candidate + self.velocity

        # Keep particle within bounds by clipping to valid range
        # This prevents particles from leaving the search space
        self.candidate = np.clip(self.candidate, self.lower, self.upper)

    def recombine(self, local_best, neighborhood_best, global_best):

        """
        Update particle velocity using PSO velocity update formula

        This is the core of PSO algorithm - updates velocity based on:
        - Current velocity (inertia term)
        - Attraction to personal best position
        - Attraction to neighborhood best position
        - Attraction to global best position

        Args:
            local_best: This particle's personal best position
            neighborhood_best: Best position among this particle's neighbors
            global_best: Best position found by any particle in the swarm
        """

        # Generate random coefficients for each attraction term, selected uniformly at random between 0 and 1
        # These add stochasticity to the velocity update
        rl = np.random.rand()  # Random coefficient for local best attraction
        rn = np.random.rand()  # Random coefficient for neighborhood best attraction
        rg = np.random.rand()  # Random coefficient for global best attraction


        # Apply PSO velocity update formula as specified in project requirements
        self.velocity = (self.inertia * self.velocity +
                        rl * self.wl * (self.candidate - local_best.candidate) +
                        rn * self.wn * (self.candidate - neighborhood_best.candidate) +
                        rg * self.wg * (self.candidate - global_best.candidate))

## 2. PSO algorithm implementation

In [4]:
class ParticleSwarmOptimizer(MetaHeuristicsAlgorithm):

    """
    Particle Swarm Optimization algorithm implementation

    Inherits from MetaHeuristicsAlgorithm to maintain compatibility with softpy.
    Manages a population of particles and orchestrates their movement through
    the search space to find optimal solutions.
    """

    def __init__(self, fitness_func, pop_size, n_neighbors, **kwargs):

        """
        Initialize the PSO algorithm

        Args:
            fitness_func: Function that evaluates particle fitness
            pop_size: Number of particles in the swarm
            n_neighbors: Number of neighbors each particle considers
            **kwargs: Additional parameters passed to particle generation
        """

        # Store algorithm parameters
        self.fitness_func = fitness_func    # Function to optimize
        self.pop_size = pop_size           # Size of particle swarm
        self.n_neighbors = n_neighbors     # Neighborhood size for each particle
        self.kwargs = kwargs               # Parameters for particle generation

        # Initialize data structures to track algorithm state
        self.population = []                # List of current particles
        self.best = []                    # Personal best positions for each particle
        self.fitness_best = np.array([])  # Personal best fitness values
        self.global_best = None           # Overall best particle found
        self.global_fitness_best = -np.inf  # Overall best fitness value

    def fit(self, n_iters):

        """
        Run the PSO algorithm for specified number of iterations

        This is the main algorithm loop that:
        1. Initializes the particle swarm
        2. Iteratively updates particles based on PSO rules
        3. Tracks best solutions found

        Args:
            n_iters: Number of iterations to run the algorithm

        Returns:
            The best particle (solution) found during optimization
        """

        # Step 1: Create initial population of random particles
        # Each particle is generated using the parameters passed in **kwargs
        self.population = [ParticleCandidate.generate(**self.kwargs) for _ in range(self.pop_size)]

        # Step 2: Initialize personal best tracking
        # Create copies of particles, not references
        # This ensures personal bests don't change when particles move
        self.best = []
        for p in self.population:
            # Create independent copy for personal best tracking
            best_copy = ParticleCandidate(p.size, p.lower, p.upper, p.candidate.copy(), p.velocity.copy())
            self.best.append(best_copy)

        # Initialize personal best fitness values to negative infinity
        # This ensures any real fitness will be better than initial value
        self.fitness_best = np.full(self.pop_size, -np.inf)

        # Step 3: Main optimization loop
        for iteration in range(n_iters):

            # Step 3a: Evaluate fitness of all particles in current population
            current_fitness = [self.fitness_func(p) for p in self.population]

            # Step 3b: Update personal bests and global best
            for i in range(self.pop_size):
                # Check if current particle achieved better fitness than its personal best
                if current_fitness[i] > self.fitness_best[i]:
                    # Update personal best fitness
                    self.fitness_best[i] = current_fitness[i]

                    # This preserves the best position even as particle continues moving
                    self.best[i] = ParticleCandidate(
                        self.population[i].size,
                        self.population[i].lower,
                        self.population[i].upper,
                        self.population[i].candidate.copy(),  # Copy position
                        self.population[i].velocity.copy()    # Copy velocity
                    )

                # Check if current particle achieved new global best
                if current_fitness[i] > self.global_fitness_best:
                    # Update global best fitness
                    self.global_fitness_best = current_fitness[i]

                    # Store copy for global best too
                    self.global_best = ParticleCandidate(
                        self.population[i].size,
                        self.population[i].lower,
                        self.population[i].upper,
                        self.population[i].candidate.copy(),
                        self.population[i].velocity.copy()
                    )

            # Step 3c: Update all particles using PSO rules
            for i in range(self.pop_size):

                # Select neighbors for current particle
                # Create list of all other particle indices (excluding current particle)
                others = [j for j in range(self.pop_size) if j != i]

                if len(others) > 0:
                    # Randomly select n_neighbors from other particles
                    # Use min() to handle case where n_neighbors > available particles
                    neighbor_idx = np.random.choice(others,
                                                  size=min(self.n_neighbors, len(others)),
                                                  replace=False)

                    # Find the neighbor with best personal fitness
                    best_n = neighbor_idx[0]  # Start with first neighbor
                    for idx in neighbor_idx:
                        # Compare fitness values to find best neighbor
                        if self.fitness_best[idx] > self.fitness_best[best_n]:
                            best_n = idx

                    # Get the personal best of the best neighbor
                    best_neighbor = self.best[best_n]
                else:
                    # Fallback: if no other particles, use global best as neighbor
                    best_neighbor = self.global_best

                # Step 3d: Apply PSO update rules to current particle
                # First: Update velocity based on attractions to best positions
                self.population[i].recombine(self.best[i], best_neighbor, self.global_best)

                # Second: Update position by adding velocity
                self.population[i].mutate()

        # Return the best solution found during optimization
        return self.global_best

## Test of the implementation

In [5]:
def simple_fitness(particle):

    """
    Simple test fitness function - quadratic with maximum at origin

    This function has its global maximum at [0, 0, ..., 0] and decreases
    quadratically as we move away from the origin. Perfect for testing
    whether PSO can find the global optimum.

    Args:
        particle: ParticleCandidate to evaluate

    Returns:
        Fitness value (higher is better)
    """

    return -np.sum(particle.candidate ** 2)

print("Testing PSO implementation...")

# Create PSO instance with test parameters
pso = ParticleSwarmOptimizer(
    fitness_func=simple_fitness,  # Function to optimize
    pop_size=15,                  # 15 particles in swarm
    n_neighbors=3,                # Each particle considers 3 neighbors
    size=2,                       # 2-dimensional problem
    lower=np.array([-10.0, -10.0]),  # Lower bounds for each dimension
    upper=np.array([10.0, 10.0])     # Upper bounds for each dimension
)

# Run optimization
result = pso.fit(n_iters=10000)

# Display results
print(f"Best solution found: {result.candidate}")
print(f"Best fitness: {pso.global_fitness_best:.6f}")
print(f"Distance from optimum [0,0]: {np.linalg.norm(result.candidate):.4f}")

Testing PSO implementation...
Best solution found: [-0.78937491  0.94114854]
Best fitness: -1.508873
Distance from optimum [0,0]: 1.2284
